# Programming Agent Memory

## Setup

In [ ]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [ ]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)
    elif message.message_type == "usage_statistics":
        # for streaming specifically, we send the final chunk that contains the usage statistics
        print(f"Usage: [{message}]")
    else:
        print(message)
    print("-----------------------------------------------------")

## Section 1: Memory Blocks

### Creating an agent

In [ ]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "The human's name is Bob the Builder."
        },
        {
          "label": "persona",
          "value": "My name is Sam, the all-knowing sentient AI."
        }
    ],
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small"
)

### Accessing blocks

In [ ]:
blocks = client.agents.blocks.list(
    agent_id=agent_state.id,
)

📝 Note: Memory blocks are returned as an unordered list and you may receive blocks in an order different than in the video

In [ ]:
blocks

[Block(value='My name is Sam, the all-knowing sentient AI.', limit=5000, name=None, is_template=False, label='persona', description=None, metadata={}, id='block-16b2ebb2-28da-4851-bf41-b42f39a74bc0', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000'),
 Block(value="The human's name is Bob the Builder.", limit=5000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-e9b860ca-e429-4b58-86fa-5f3d9a1bd577', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')]

In [ ]:
# Note: Replace the block_id with the id from the cell above.
block_id='block-16b2ebb2-28da-4851-bf41-b42f39a74bc0'

In [ ]:
client.blocks.retrieve(block_id)

Block(value='My name is Sam, the all-knowing sentient AI.', limit=5000, name=None, is_template=False, label='persona', description=None, metadata={}, id='block-16b2ebb2-28da-4851-bf41-b42f39a74bc0', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

In [ ]:
human_block = client.agents.blocks.retrieve(
    agent_id=agent_state.id,
    block_label="human",
)
human_block

Block(value="The human's name is Bob the Builder.", limit=5000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-e9b860ca-e429-4b58-86fa-5f3d9a1bd577', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

### Accessing block prompt template

In [ ]:
client.agents.core_memory.retrieve(
    agent_id=agent_state.id
).prompt_template

'{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}'

## Section 2: Accessing `AgentState` with Tools

### Creating tools

In [ ]:
def get_agent_id(agent_state: "AgentState"):
    """
    Query your agent ID field
    """
    return agent_state.id

In [ ]:
get_id_tool = client.tools.upsert_from_function(func=get_agent_id)

### Creating agents that use tools

In [ ]:
agent_state = client.agents.create(
    memory_blocks=[],
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small",
    tool_ids=[get_id_tool.id]
)

In [ ]:
response_stream = client.agents.messages.create_stream(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What is your agent id?"
        }
    ]
)

for chunk in response_stream:
    print_message(chunk)

🧠 Reasoning: User asked for agent ID. I need to retrieve that information for them.
-----------------------------------------------------
🔧 Tool Call: get_agent_id
{
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: agent-87225fd2-23a3-4c13-b686-a367e6ccc833
-----------------------------------------------------
🧠 Reasoning: I retrieved the agent ID successfully. Now, I will share it with the user.
-----------------------------------------------------
🤖 Agent: My agent ID is: agent-87225fd2-23a3-4c13-b686-a367e6ccc833.
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=105 prompt_tokens=4347 total_tokens=4452 step_count=2 steps_messages=None run_ids=None]
-----------------------------------------------------


## Section 3: Custom Task Queue Memory

### Creating custom memory management tools

In [ ]:
def task_queue_push(agent_state: "AgentState", task_description: str):
    """
    Push to a task queue stored in core memory.

    Args:
        task_description (str): A description of the next task you must accomplish.

    Returns:
        Optional[str]: None is always returned as this function
        does not produce a response.
    """

    from letta_client import Letta
    import json

    client = Letta(base_url="http://localhost:8283")

    block = client.agents.blocks.retrieve(
        agent_id=agent_state.id,
        block_label="tasks",
    )
    tasks = json.loads(block.value)
    tasks.append(task_description)

    # update the block value
    client.agents.blocks.modify(
        agent_id=agent_state.id,
        value=json.dumps(tasks),
        block_label="tasks"
    )
    return None

In [ ]:
def task_queue_pop(agent_state: "AgentState"):
    """
    Get the next task from the task queue

    Returns:
        Optional[str]: Remaining tasks in the queue
    """

    from letta_client import Letta
    import json

    client = Letta(base_url="http://localhost:8283")

    # get the block
    block = client.agents.blocks.retrieve(
        agent_id=agent_state.id,
        block_label="tasks",
    )
    tasks = json.loads(block.value)
    if len(tasks) == 0:
        return None
    task = tasks[0]

    # update the block value
    remaining_tasks = json.dumps(tasks[1:])
    client.agents.blocks.modify(
        agent_id=agent_state.id,
        value=remaining_tasks,
        block_label="tasks"
    )
    return f"Remaining tasks {remaining_tasks}"

### Upserting tools into Letta

In [ ]:
task_queue_pop_tool = client.tools.upsert_from_function(
    func=task_queue_pop
)
task_queue_push_tool = client.tools.upsert_from_function(
    func=task_queue_push
)

In [ ]:
import json

task_agent = client.agents.create(
    system=open("task_queue_system_prompt.txt", "r").read(),
    memory_blocks=[
        {
          "label": "tasks",
          "value": json.dumps([])
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small",
    tool_ids=[task_queue_pop_tool.id, task_queue_push_tool.id],
    include_base_tools=False,
    tools=["send_message"]
)

In [ ]:
[tool.name for tool in task_agent.tools]

['send_message', 'task_queue_push', 'task_queue_pop']

In [ ]:
client.agents.blocks.retrieve(task_agent.id, block_label="tasks").value

'[]'

### Using task agent

In [ ]:
response_stream = client.agents.messages.create_stream(
    agent_id=task_agent.id,
    messages=[
        {
            "role": "user",
            "content": "Add 'start calling me Charles' and "
            + "'tell me a haiku about my name' as two seperate tasks."
        }
    ]
)

for chunk in response_stream:
    print_message(chunk)

🧠 Reasoning: User requested to start calling them Charles. Adding to task queue.
-----------------------------------------------------
🔧 Tool Call: task_queue_push
{
  "task_description": "Start calling the user Charles.",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: None
-----------------------------------------------------
🧠 Reasoning: Next task is to create a haiku about the name Charles. Adding this to the queue now.
-----------------------------------------------------
🔧 Tool Call: task_queue_push
{
  "task_description": "Tell a haiku about the user name Charles.",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: None
-----------------------------------------------------
🧠 Reasoning: Clearing the first task from the queue.
-----------------------------------------------------
🔧 Tool Call: task_queue_pop
{
  "request_heartbeat": true
}
--------------------------------------------

In [ ]:
response_stream = client.agents.messages.create_stream(
    agent_id=task_agent.id,
    messages=[
        {
            "role": "user",
            "content": "Complete your tasks"
        }
    ]
)

for chunk in response_stream:
    print_message(chunk)

🧠 Reasoning: User wants to see task completion. Clarifying that tasks are done.
-----------------------------------------------------
🤖 Agent: All tasks are complete, Charles! If you have more for me, feel free to share!
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=57 prompt_tokens=2128 total_tokens=2185 step_count=1 steps_messages=None run_ids=None]
-----------------------------------------------------


### Retrieving task list

In [ ]:
client.agents.blocks.retrieve(block_label="tasks", agent_id=task_agent.id).value

'[]'